In [ ]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('/content/TBreport_SDG381382.csv')
df2 = pd.read_csv('/content/tb_2022-09-30.csv')
df3 = pd.read_csv('/content/tbhiv_2022-09-27.csv')
df4 = pd.read_csv('/content/tx_2022-08-29.csv')

# Define column mappings for each DataFrame
df1_columns = {
    'Country Name': 'country',
    'year': 'year',
    'sdg381': 'sdg381',
    'SDG 3.8.2': 'sdg_3_8_2',
    'WB income group classification': 'wb_income_group'
}

df2_renaming = {
    'iso3': 'region',
    'e.pop.15plus': 'total_population',
    'e.pop.num': 'cases_per_100k',
    'pop': 'total_cases',
    'e.pop.m04': 'm_0to14_cases',
    'e.pop.m514': 'm_0to14_cases',
    'e.pop.m014': 'm_0to14_cases',
    'e.pop.m1524': 'm_15to34_cases',
    'e.pop.m2534': 'm_15to34_cases',
    'e.pop.m3544': 'm_35to54_cases',
    'e.pop.m4554': 'm_35to54_cases',
    'e.pop.m5564': 'm_55plus_cases',
    'e.pop.m65': 'm_55plus_cases',
    'e.pop.m15plus': 'total_male_cases',
    'e.pop.f04': 'f_0to14_cases',
    'e.pop.f514': 'f_0to14_cases',
    'e.pop.f014': 'f_0to14_cases',
    'e.pop.f1524': 'f_15to34_cases',
    'e.pop.f2534': 'f_15to34_cases',
    'e.pop.f3544': 'f_35to54_cases',
    'e.pop.f4554': 'f_35to54_cases',
    'e.pop.f5564': 'f_55plus_cases',
    'e.pop.f65': 'f_55plus_cases',
    'e.pop.f15plus': 'total_female_cases'
}

df3_columns = {
    'iso3': 'region',
    'year': 'year',
    'g.whoregion': 'g_whoregion',
    'newinc': 'newinc',
    'c.newinc': 'c_newinc',
    'newrel.hivpos': 'newrel_hivpos',
    'newrel.hivneg': 'newrel_hivneg',
    'newrel.unkhiv': 'newrel_unkhiv'
}

df4_columns = {
    'country': 'country',
    'year': 'year',
    'iso2': 'iso2',
    'iso3': 'region',
    'iso.numeric': 'iso_numeric',
    'g.whoregion': 'g_whoregion',
    'tx.new': 'tx_new',
    'tx.all': 'tx_all',
    'tx.fail': 'tx_fail',
    'tx.succ': 'tx_succ',
    'tx.died': 'tx_died',
    'tx.def': 'tx_def'
}

# Rename the columns
df1 = df1.rename(columns=df1_columns)
df2 = df2.rename(columns=df2_renaming)
df3 = df3.rename(columns=df3_columns)
df4 = df4.rename(columns=df4_columns)

# Add missing columns if necessary
for i, df in enumerate([df1, df2, df3, df4], 1):
    for col in ['country', 'year']:
        if col not in df.columns:
            print(f"Adding missing column '{col}' to df{i}")
            df[col] = None  # Add placeholder if column is missing

# Drop columns with more than 50% null values
df1 = df1.dropna(thresh=len(df1) * 0.5, axis=1)
df2 = df2.dropna(thresh=len(df2) * 0.5, axis=1)
df3 = df3.dropna(thresh=len(df3) * 0.5, axis=1)
df4 = df4.dropna(thresh=len(df4) * 0.5, axis=1)

# Ensure 'year' is numeric in all DataFrames
for df in [df1, df2, df3, df4]:
    df['year'] = pd.to_numeric(df['year'], errors='coerce', downcast='integer')

# Merge the DataFrames on 'country' and 'year', ignoring 'country' for df3
dfs_to_merge = []

for i, df in enumerate([df1, df2, df3, df4], 1):
    if 'country' in df.columns:
        dfs_to_merge.append(df)
    else:
        # Include 'year' for merging if 'country' is not present
        dfs_to_merge.append(df.drop_duplicates(subset=['year']))

# Perform merging step by step
merged_df = dfs_to_merge[0]
for df in dfs_to_merge[1:]:
    merge_keys = ['year']
    if 'country' in merged_df.columns and 'country' in df.columns:
        merge_keys = ['country', 'year']
    merged_df = merged_df.merge(df, on=merge_keys, how='outer')

# Drop duplicate 'country' columns (if any)
merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('country') | (merged_df.columns == 'country')]

# Add additional columns (placeholders if missing)
additional_columns = [
    'recovery_rate', 'economic_impact', 'case_severity', 'time_interval',
    'prevelance_detection', 'prevelance_per_100k_ex_HIV',
    'total_prevelance_ex_HIV', 'prevelance_per_100k_in_HIV',
    'total_prevelance_in_HIV', 'death_detection', 'incidence_per_100k',
    'total_incidence', 'incidence_per_100k_pos_HIV',
    'total_incidence_pos_HIV', 'total_detection_rate'
]

# Add missing columns with None values as placeholders
for col in additional_columns:
    if col not in merged_df.columns:
        merged_df[col] = None  # Add the column if it's missing

# Display the final merged DataFrame with all columns
print("Final merged DataFrame with all columns:")
merged_df[['country', 'year'] + list(df2_renaming.values()) + additional_columns].head(500)


Adding missing column 'country' to df3
Final merged DataFrame with all columns:


,country,year,region,total_population,cases_per_100k,total_cases,m_0to14_cases,m_0to14_cases,m_0to14_cases,m_0to14_cases,...,prevelance_per_100k_ex_HIV,total_prevelance_ex_HIV,prevelance_per_100k_in_HIV,total_prevelance_in_HIV,death_detection,incidence_per_100k,total_incidence,incidence_per_100k_pos_HIV,total_incidence_pos_HIV,total_detection_rate
0,Afghanistan,1980.0,NaN,6746173.0,12486631.0,12486631.0,1197631.0,1717304.0,2914935.0,1197631.0,...,None,None,None,None,None,None,None,None,None,None
1,Afghanistan,1981.0,NaN,6005396.0,11155195.0,11155195.0,1071677.0,1544152.0,2615829.0,1071677.0,...,None,None,None,None,None,None,None,None,None,None
2,Afghanistan,1982.0,NaN,5412368.0,10088289.0,10088289.0,969827.0,1406153.0,2375980.0,969827.0,...,None,None,None,None,None,None,None,None,None,None
3,Afghanistan,1983.0,NaN,5323322.0,9951449.0,9951449.0,956867.0,1395626.0,2352493.0,956867.0,...,None,None,None,None,None,None,None,None,None,None
4,Afghanistan,1984.0,NaN,5463882.0,10243686.0,10243686.0,984469.0,1445848.0,2430317.0,984469.0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Australia,2013.0,AUS,18737735.0,23111782.0,23111782.0,781052.0,1464166.0,2245218.0,781052.0,...,None,None,None,None,None,None,None,None,None,None
496,Australia,2014.0,AUS,19031157.0,23469579.0,23469579.0,790586.0,1487980.0,2278566.0,790586.0,...,None,None,None,None,None,None,None,None,None,None
497,Australia,2015.0,AUS,19318080.0,23820236.0,23820236.0,798314.0,1513169.0,2311483.0,798314.0,...,None,None,None,None,None,None,None,None,None,None
498,Australia,2016.0,AUS,19625315.0,24195701.0,24195701.0,805987.0,1540668.0,2346655.0,805987.0,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('/content/TBreport_SDG381382.csv')
df2 = pd.read_csv('/content/tb_2022-09-30.csv')
df3 = pd.read_csv('/content/tbhiv_2022-09-27.csv')
df4 = pd.read_csv('/content/tx_2022-08-29.csv')

# Define column mappings for each DataFrame
df1_columns = {
    'Country Name': 'country',
    'year': 'year',
    'sdg381': 'sdg381',
    'SDG 3.8.2': 'sdg_3_8_2',
    'WB income group classification': 'wb_income_group'

}

df2_renaming = {
    'iso3': 'region',
    'e.pop.15plus': 'total_population',
    'e.pop.num': 'cases_per_100k',
    'pop': 'total_cases',
    'e.pop.m04': 'm_0to14_cases',
    'e.pop.m514': 'm_0to14_cases',
    'e.pop.m014': 'm_0to14_cases',
    'e.pop.m1524': 'm_15to34_cases',
    'e.pop.m2534': 'm_15to34_cases',
    'e.pop.m3544': 'm_35to54_cases',
    'e.pop.m4554': 'm_35to54_cases',
    'e.pop.m5564': 'm_55plus_cases',
    'e.pop.m65': 'm_55plus_cases',
    'e.pop.m15plus': 'total_male_cases',
    'e.pop.f04': 'f_0to14_cases',
    'e.pop.f514': 'f_0to14_cases',
    'e.pop.f014': 'f_0to14_cases',
    'e.pop.f1524': 'f_15to34_cases',
    'e.pop.f2534': 'f_15to34_cases',
    'e.pop.f3544': 'f_35to54_cases',
    'e.pop.f4554': 'f_35to54_cases',
    'e.pop.f5564': 'f_55plus_cases',
    'e.pop.f65': 'f_55plus_cases',
    'e.pop.f15plus': 'total_female_cases'
}

df3_columns = {
    'iso3': 'code',
    'year': 'year',
    'g.whoregion': 'region',
    'newinc': 'newinc',
    'c.newinc': 'c_newinc',
    'newrel.hivpos': 'newrel_hivpos',
    'newrel.hivneg': 'newrel_hivneg',
    'newrel.unkhiv': 'newrel_unkhiv'
}

df4_columns = {
    'country': 'country',
    'year': 'year',
    'iso2': 'iso2',
    'iso3': 'region',
    'iso.numeric': 'iso_numeric',
    'g.whoregion': 'g_whoregion',
    'tx.new': 'tx_new',
    'tx.all': 'tx_all',
    'tx.fail': 'tx_fail',
    'tx.succ': 'tx_succ',
    'tx.died': 'tx_died',
    'tx.def': 'tx_def'
}

# Rename the columns
df1 = df1.rename(columns=df1_columns)
df2 = df2.rename(columns=df2_renaming)
df3 = df3.rename(columns=df3_columns)
df4 = df4.rename(columns=df4_columns)

# Add missing columns if necessary
for i, df in enumerate([df1, df2, df3, df4], 1):
    for col in ['country', 'year']:
        if col not in df.columns:
            print(f"Adding missing column '{col}' to df{i}")
            df[col] = None  # Add placeholder if column is missing

# Drop columns with more than 50% null values
df1 = df1.dropna(thresh=len(df1) * 0.5, axis=1)
df2 = df2.dropna(thresh=len(df2) * 0.5, axis=1)
df3 = df3.dropna(thresh=len(df3) * 0.5, axis=1)
df4 = df4.dropna(thresh=len(df4) * 0.5, axis=1)

# Ensure 'year' is numeric in all DataFrames
for df in [df1, df2, df3, df4]:
    df['year'] = pd.to_numeric(df['year'], errors='coerce', downcast='integer')

# Merge the DataFrames on 'country' and 'year', ignoring 'country' for df3
dfs_to_merge = []

for i, df in enumerate([df1, df2, df3, df4], 1):
    if 'country' in df.columns:
        dfs_to_merge.append(df)
    else:
        # Include 'year' for merging if 'country' is not present
        dfs_to_merge.append(df.drop_duplicates(subset=['year']))

# Perform merging step by step
merged_df = dfs_to_merge[0]
for df in dfs_to_merge[1:]:
    merge_keys = ['year']
    if 'country' in merged_df.columns and 'country' in df.columns:
        merge_keys = ['country', 'year']
    merged_df = merged_df.merge(df, on=merge_keys, how='outer')

# Drop duplicate 'country' columns (if any)
merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('country') | (merged_df.columns == 'country')]

# Add additional columns (placeholders if missing)
additional_columns = [
    'recovery_rate', 'economic_impact', 'case_severity', 'time_interval',
    'prevelance_detection', 'prevelance_per_100k_ex_HIV',
    'total_prevelance_ex_HIV', 'prevelance_per_100k_in_HIV',
    'total_prevelance_in_HIV', 'death_detection', 'incidence_per_100k',
    'total_incidence', 'incidence_per_100k_pos_HIV',
    'total_incidence_pos_HIV', 'total_detection_rate'
]

# Add missing columns with None values as placeholders
for col in additional_columns:
    if col not in merged_df.columns:
        merged_df[col] = None  # Add the column if it's missing

# Populate the columns with dummy or calculated values (this can be adjusted as per your actual calculations)
# For demonstration, I will use simple dummy values or placeholder calculations.

# Ensure 'df2' contains necessary columns for calculations
merged_df['economic_impact'] = merged_df['total_cases'] * 0.5 if 'total_cases' in df2.columns else None
merged_df['case_severity'] = merged_df['total_deaths'] / merged_df['total_cases'] if 'total_deaths' in df2.columns and 'total_cases' in df2.columns else None
merged_df['time_interval'] = (merged_df['year'] - merged_df['year'].min()) if 'year' in df2.columns else None
merged_df['prevelance_detection'] = merged_df['total_cases'] / merged_df['total_population'] * 100000 if 'total_cases' in df2.columns and 'total_population' in df2.columns else None
merged_df['prevelance_per_100k_ex_HIV'] = merged_df['prevelance_detection'] * 0.8  # Assuming 80% of cases are not related to HIV
merged_df['total_prevelance_ex_HIV'] = merged_df['total_cases'] * 0.8 if 'total_cases' in df2.columns else None
merged_df['prevelance_per_100k_in_HIV'] = merged_df['prevelance_detection'] * 0.2  # Assuming 20% of cases are related to HIV
merged_df['total_prevelance_in_HIV'] = merged_df['total_cases'] * 0.2 if 'total_cases' in df2.columns else None
merged_df['death_detection'] = merged_df['total_deaths'] / merged_df['total_population'] * 100000 if 'total_deaths' in df2.columns and 'total_population' in df2.columns else None

merged_df['total_detection_rate'] = merged_df['total_cases'] / merged_df['total_population'] if 'total_cases' in df2.columns and 'total_population' in df2.columns else None

# Show the first few rows of the merged dataset
merged_df[['country', 'year'] + list(df2_renaming.values()) + additional_columns].head(500)



Adding missing column 'country' to df3


,country,year,region,total_population,cases_per_100k,total_cases,m_0to14_cases,m_0to14_cases,m_0to14_cases,m_0to14_cases,...,prevelance_per_100k_ex_HIV,total_prevelance_ex_HIV,prevelance_per_100k_in_HIV,total_prevelance_in_HIV,death_detection,incidence_per_100k,total_incidence,incidence_per_100k_pos_HIV,total_incidence_pos_HIV,total_detection_rate
0,Afghanistan,1980.0,NaN,6746173.0,12486631.0,12486631.0,1197631.0,1717304.0,2914935.0,1197631.0,...,148073.653018,9989304.8,37018.413254,2497326.2,None,None,None,None,None,1.850921
1,Afghanistan,1981.0,NaN,6005396.0,11155195.0,11155195.0,1071677.0,1544152.0,2615829.0,1071677.0,...,148602.290340,8924156.0,37150.572585,2231039.0,None,None,None,None,None,1.857529
2,Afghanistan,1982.0,NaN,5412368.0,10088289.0,10088289.0,969827.0,1406153.0,2375980.0,969827.0,...,149114.605659,8070631.2,37278.651415,2017657.8,None,None,None,None,None,1.863933
3,Afghanistan,1983.0,NaN,5323322.0,9951449.0,9951449.0,956867.0,1395626.0,2352493.0,956867.0,...,149552.463668,7961159.2,37388.115917,1990289.8,None,None,None,None,None,1.869406
4,Afghanistan,1984.0,NaN,5463882.0,10243686.0,10243686.0,984469.0,1445848.0,2430317.0,984469.0,...,149984.000387,8194948.8,37496.000097,2048737.2,None,None,None,None,None,1.874800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Australia,2013.0,AUS,18737735.0,23111782.0,23111782.0,781052.0,1464166.0,2245218.0,781052.0,...,98674.816353,18489425.6,24668.704088,4622356.4,None,None,None,None,None,1.233435
496,Australia,2014.0,AUS,19031157.0,23469579.0,23469579.0,790586.0,1487980.0,2278566.0,790586.0,...,98657.497282,18775663.2,24664.374320,4693915.8,None,None,None,None,None,1.233219
497,Australia,2015.0,AUS,19318080.0,23820236.0,23820236.0,798314.0,1513169.0,2311483.0,798314.0,...,98644.320761,19056188.8,24661.080190,4764047.2,None,None,None,None,None,1.233054
498,Australia,2016.0,AUS,19625315.0,24195701.0,24195701.0,805987.0,1540668.0,2346655.0,805987.0,...,98630.573828,19356560.8,24657.643457,4839140.2,None,None,None,None,None,1.232882
